In [44]:
import pandas as pd
import numpy as np
from IPython.display import display, display_pretty, Javascript, HTML
from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt

# Show charts in notebook
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [12]:
# Load 2014 and 2015 survey household data
hh14 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-households.xlsx', sheet='Data')
hh15 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2015\Data\Household\1_PSRC2015_HH_2015-07-27_V1.2.xlsx', sheet='Data')

In [5]:
# Load 2014 and 2015 survey household data
person14 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-persons.xlsx', sheet='Data')
person15 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2015\Data\Person\3_PSRC2015_Person_2015-07-27_V1.2.xlsx', sheet='Data')

In [20]:
len(person15)

4786

In [14]:
# Load the combined 2014-2015 dataset to get expansion weights
hh14_15 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Combined\1_Combined_HH_2015-11-17.xlsx', sheet='Data')

In [16]:
# Select only the 2015 household records
hh14_15=hh14_15[hh14_15['datasource'] == 'hts2015']
len(hh14_15)

2428

In [17]:
person15 = pd.merge(person15,hh14_15[['hhid','expwt_h1415']],on='hhid')

In [28]:
per14 = person14
per15 = person15

In [41]:
df1 = per14
df2 = per15

In [31]:
expwt14 = 'expwt_final'
expwt15 = 'expwt_h1415'    # No weights yet so just using arbitrary column to sum up counts

person14.year = '2014'
person15.year = '2015'

In [48]:
labels = {
1: "<5",
2: "5-11",
3: "12-15",
4: "16-17",
5: "18-24",
6: "25-34",
7: "35-44",
8: "45-54",
9: "55-64",
10: "65-74",
11: "75-84",
12: "85+"}


df1['age'] = ([labels[x] for x in df1['age']])
df2['age'] = ([labels[x] for x in df2['age']])

In [60]:
labels = {
0: "No answer",
1: "Employed full-time",
2: "Employed part-time",
3: "Self-employed",
4: "Unpaid volunteed",
5: "Homemaker",
6: "Retired",
7: "Not employed"
}

df1['employment'].fillna(0, inplace=True)
df2['employment'].fillna(0, inplace=True)
df1['employment'] = ([labels[x] for x in df1['employment']])
df2['employment'] = ([labels[x] for x in df2['employment']])

In [ ]:
labels = {
1: "male",
2: "female"
}

df1['gender'] = ([labels[x] for x in df1['gender']])
df2['gender'] = ([labels[x] for x in df2['gender']])

In [69]:
fieldname = 'student'
labels = {
0: "No response",
1: "Not a student",
2: "Full-time student",
3: "Part-time student",
4: "Vocational/technical student"
}

df1[fieldname].fillna(0, inplace=True)
df2[fieldname].fillna(0, inplace=True)
df1[fieldname] = ([labels[x] for x in df1[fieldname]])
df2[fieldname] = ([labels[x] for x in df2[fieldname]])

In [70]:
fieldname = 'education'
labels = {
0: "No response",
1: "Less than high school",
2: "High school graduate",
3: "Some college",
4: "Vocational/technical training",
5: "Associates degree",
6: "Bachelor degree",
7: "Graduate/post-graduate degree"
}

df1[fieldname].fillna(0, inplace=True)
df2[fieldname].fillna(0, inplace=True)
df1[fieldname] = ([labels[x] for x in df1[fieldname]])
df2[fieldname] = ([labels[x] for x in df2[fieldname]])

In [25]:
def df_sample(df1, df2, field, agg1, agg2, report_avg=True, print_df=False):
    '''Compare samples from two surveys in the same dataframe'''
    df = pd.DataFrame()
    df[df1.year + ' Samples'] = df1.groupby(field).count()[agg1]
    df[df2.year + ' Samples'] = df2.groupby(field).count()[agg2]
    df[" "*5] = ""    # empty col to separate samples from shares
    df[df1.year + ' Shares'] = (df1.groupby(field).count()[agg1]/df1[agg1].count()).round(2)
    df[df2.year + ' Shares'] = (df2.groupby(field).count()[agg2]/df2[agg2].count()).round(2)

    if print_df:
        print df  

        print "---------" * 10

        if report_avg:
            print "Sample average " + df1.year + ": " + str(np.round(df1[field].mean(),2))
            print "Sample average " + df2.year + ": " + str(np.round(df2[field].mean(),2))
            
    else:
        return df

In [26]:
def df_weighted(df1, df2, field, agg1, agg2):
    '''Compare weighted results from two data sources'''
    df = pd.DataFrame()
    df[df1.year] = df1.groupby(field).sum()[agg1]
    df[df2.year] = df2.groupby(field).sum()[agg2]
    df[" "*5] = ""    # empty col to separate samples from shares
    df[df1.year + ' Shares'] = (df1.groupby(field).sum()[agg1]/df1[agg1].sum()).round(2)
    df[df2.year + ' Shares'] = (df2.groupby(field).sum()[agg2]/df2[agg2].sum()).round(2)
    
    return df

In [27]:
def df_mean(df, fieldname):
    '''Calculate mean for categorical dataframes'''
    mean = {}
    mean[df1.year] = (df[df1.year]*df.index).sum()/(df[df1.year].sum())
    mean[df2.year] = (df[df2.year]*df.index).sum()/(df[df2.year].sum())
    print 'Mean - ' + fieldname
    print df1.year + ': ' + str(mean[df1.year])
    print df2.year + ': ' + str(mean[df2.year])

** Age Category**

In [49]:
fieldname = 'age'
df = df_weighted(per14, per15, fieldname, expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
age,,,,,
12-15,184165.9639,190764.542597,,0.05,0.05
16-17,81213.2928,75486.747351,,0.02,0.02
18-24,202720.4296,195725.567723,,0.06,0.05
25-34,543963.5019,556810.857804,,0.16,0.15
35-44,506535.8516,560040.845658,,0.15,0.15
45-54,526958.4817,511913.650486,,0.15,0.14
5-11,306435.4751,376064.417550,,0.09,0.10
55-64,489038.0149,512457.194449,,0.14,0.14
65-74,272537.3586,277338.494330,,0.08,0.08


In [51]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Gender **

In [53]:
fieldname = 'gender'
df = df_weighted(per14, per15, fieldname, expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
gender,,,,,
1,1655805.1547,1944793.844615,,0.48,0.53
2,1808730.4897,1741540.554044,,0.52,0.47


In [54]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Employment **

In [79]:
fieldname = 'Employment'
df = df_weighted(per14, per15, 'employment', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
employment,,,,,
Employed full-time,1295664.5362,1355625.408051,,0.37,0.37
Employed part-time,349441.4752,352876.335554,,0.10,0.10
Homemaker,172112.6044,220030.630080,,0.05,0.06
No answer,793910.5155,926148.199929,,0.23,0.25
Not employed,229365.2958,162321.715023,,0.07,0.04
Retired,385945.3978,419060.888294,,0.11,0.11
Self-employed,213930.6533,228674.107502,,0.06,0.06
Unpaid volunteed,24165.1662,21597.114227,,0.01,0.01


In [80]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Student Status **

In [81]:
fieldname = 'Student Status'
df = df_weighted(per14, per15, 'student', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
student,,,,,
Full-time student,87932.0643,56039.267053,,0.03,0.02
No response,793910.5155,926148.199929,,0.23,0.25
Not a student,2437268.1698,2571752.831632,,0.70,0.70
Part-time student,145424.8948,132394.100045,,0.04,0.04


In [82]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Education **

In [83]:
fieldname = 'Education'
df = df_weighted(per14, per15, 'education', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
education,,,,,
Associates degree,237721.3285,217968.862305,,0.07,0.06
Bachelor degree,873411.3585,900925.500289,,0.25,0.24
Graduate/post-graduate degree,554493.4891,632994.698973,,0.16,0.17
High school graduate,308881.4595,325176.887280,,0.09,0.09
Less than high school,71289.6480,57293.361377,,0.02,0.02
No response,793910.5155,926148.199929,,0.23,0.25
Some college,475639.8167,496970.734301,,0.14,0.13
Vocational/technical training,149188.0286,128856.154206,,0.04,0.03


In [84]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Smartphone **

In [85]:
fieldname = 'Smartphone'
df = df_weighted(per14, per15, 'smartphone', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
smartphone,,,,,
1,1951882.0535,2148729.210594,,0.56,0.58
2,133606.7094,95156.605950,,0.04,0.03
3,666349.6588,591787.129537,,0.19,0.16


In [86]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** License **

In [87]:
fieldname = 'License'
df = df_weighted(per14, per15, 'license', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
license,,,,,
1,2525601.1818,2623833.630139,,0.73,0.71
2,226237.2399,211839.315943,,0.07,0.06


In [88]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Toll Frequency **

In [89]:
fieldname = 'Toll Frequency'
df = df_weighted(per14, per15, 'tollfreq', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
tollfreq,,,,,
1,70595.2680,66722.403237,,0.02,0.02
2,58957.2288,65333.835142,,0.02,0.02
3,52248.0384,85117.005446,,0.02,0.02
4,269844.7279,323073.857154,,0.08,0.09
5,1078156.4438,1229842.628754,,0.31,0.33
6,1108222.0112,973030.224350,,0.32,0.26
7,113814.7036,92552.991998,,0.03,0.03


In [90]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** School Frequency **

In [91]:
fieldname = 'School Frequency'
df = df_weighted(per14, per15, 'school_freq', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
school_freq,,,,,
1,20178.9670,15066.198112,,0.01,0.00
2,531246.5246,593991.101407,,0.15,0.16
3,28299.8459,23250.160543,,0.01,0.01
4,27230.4133,26639.862624,,0.01,0.01
5,32456.6742,18575.806290,,0.01,0.01
6,17459.3016,8974.652654,,0.01,0.00
7,14710.8742,9771.494654,,0.00,0.00
8,67337.9850,59262.109693,,0.02,0.02


In [92]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Use of transit **

In [93]:
fieldname = 'Transt Use Frequency'
df = df_weighted(per14, per15, 'transit_freq', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
transit_freq,,,,,
1,69823.1002,74772.533183,,0.02,0.02
2,213975.6761,254900.178101,,0.06,0.07
3,204018.8176,174250.758600,,0.06,0.05
4,88571.6358,85053.393613,,0.03,0.02
5,358390.4855,451086.589302,,0.10,0.12
6,703590.4876,744995.479130,,0.20,0.20
7,1604069.6579,1617442.974301,,0.46,0.44


In [94]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Bike Frequency **

In [98]:
fieldname = 'Bike Use Frequency'
df = df_weighted(per14, per15, 'bike_freq', expwt14, expwt15)
df.drop(df.index[df.index.get_loc(-99)],inplace=True)   # drop no-response field
df

,2014,2015,,2014 Shares,2015 Shares
bike_freq,,,,,
1,33822.3347,25523.745683,,0.01,0.01
2,55796.3914,48883.161486,,0.02,0.01
3,188977.1983,235568.810954,,0.05,0.06
4,109656.1765,199154.099954,,0.03,0.05
5,245687.1710,221274.736570,,0.07,0.06
6,647664.5590,640956.058536,,0.19,0.17
7,1875764.3387,2031141.293047,,0.54,0.55


In [99]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Walk Frequency **

In [101]:
fieldname = 'Walk Frequency'
df = df_weighted(per14, per15, 'walk_freq', expwt14, expwt15)
df.drop(df.index[df.index.get_loc(-99)],inplace=True)   # drop no-response field
df

,2014,2015,,2014 Shares,2015 Shares
walk_freq,,,,,
1,406400.5191,430207.228748,,0.12,0.12
2,342140.8455,411610.148506,,0.10,0.11
3,847660.7077,852660.035459,,0.24,0.23
4,321458.4548,330882.093467,,0.09,0.09
5,425238.7992,427210.885529,,0.12,0.12
6,299501.8353,286403.630310,,0.09,0.08
7,514967.0080,663527.884209,,0.15,0.18


In [102]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Transit Subsidy **

** Telecommute Frequency **

** Work Benefits: Flextime**

**Work Benefits: Compressed Work Week**

**Work Benefits: Free Parking**

**Work Benefits: Transit Pass**

** Work Benefits: Other Commute Subsidies**

** Would Carpool if: Gas Cost OVer $5/gal**

**Would Carpool if: Price of Parking Increased 50%**

**Would Carpool if: tolls were more than $5+ per trip **

** Would Carpool if: HOV saved 10 min. per trip  **

**Would Carpool if: transit saved 10+ min. per trip (over SOV)**

**Would Carpool if: other reason**

**Would Carpool if: none of those would affect me**

** Has Car2Go Membership **

**Car2Go Use Frequency**

** RelayRides Membership **

**ZipCar Membership**

**No CarShare Membership**

**Pronto Membership**

** Uber Usage Frequnecy **

In [105]:
# fieldname = 'Smartphone Type'
df = df_weighted(per14, per15, 'share_freq_uberx', expwt14, expwt15)
# df.drop(df.index[df.index.get_loc(-99)],inplace=True)   # drop no-response field
df

KeyError: 'share_freq_uberx'